In [ ]:
import torch as th
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchmetrics import MetricCollection, MeanAbsoluteError, R2Score
import pytorch_lightning as pl

import pandas as pd
import numpy as np
from typing import Any, Optional

# mlflow server --host 127.0.0.1 --port 8080

class MyLightningModule(pl.LightningModule):
    def __init__(self, body, learning_rate):
        super().__init__()

        self.learning_rate = learning_rate

        self.model = body

        self.criterion = nn.MSELoss()

        self.metrics = MetricCollection({
            "MeanAbsoluteError": MeanAbsoluteError(),
            "R2Score": R2Score() 
        })

    # Хочу вручную задать те гипперпараметры которые зафиксиру в ране
    def on_fit_start(self):
        params = {
            "lr": self.learning_rate,
            "optimizer": self.configure_optimizers().__class__.__name__,
            "model": str(self.model),
        }

        for key, item in params.items():
            self.logger.experiment.log_param(self.logger.run_id, key, item)
    
    # Хочу артифактом добавить лучший чекпоинт модели
    def on_train_end(self):
        best_ckpt = self.trainer.checkpoint_callback.best_model_path
        if best_ckpt:
            self.logger.experiment.log_artifact(
                run_id=self.logger.run_id,
                local_path=best_ckpt
            )

    def forward(self, x: th.Tensor) -> th.Tensor:
        return self.model(x).flatten()

    def training_step(self, batch: Any, batch_idx: int) -> th.Tensor:
        return self.__step(batch, batch_idx)

    def validation_step(self, batch: Any, batch_idx: int) -> None:
        return self.__step(batch, batch_idx)

    def test_step(self, batch: Any, batch_idx: int) -> None:
        return self.__step(batch, batch_idx)

    def __step(self, batch: Any, batch_idx: int) -> th.Tensor:
        X, y = batch
        y_pred = self(X)

        loss = self.criterion(y_pred, y)

        self.log("loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log_dict(
            {
                key: item for key, item in self.metrics(y_pred, y).items()
            }, 
            on_step=False, 
            on_epoch=True, 
            prog_bar=True
        )

        return loss

    def configure_optimizers(self) -> th.optim.Optimizer:
        return optim.AdamW(self.parameters(), lr=self.learning_rate)


In [5]:
import mlflow
import subprocess

if 'mlflow_process' in locals() and mlflow_process.poll() is None:
    mlflow_process.terminate()
    mlflow_process.wait()
    print("Остановлен предыдущий процесс MLflow UI.")

In [6]:
port = 8080
mlflow_process = subprocess.Popen(["mlflow", "ui", "--port", str(port)])
print(f"MLflow UI запущен с PID (ID процесса): {mlflow_process.pid}")
mlflow_url = f"http://localhost:{port}"
mlflow.set_tracking_uri("file:./mlruns")
print(mlflow_url)

MLflow UI запущен с PID (ID процесса): 10504
http://localhost:8080


In [7]:
class DatasetFloors(Dataset):
    def __init__(self, data: pd.DataFrame, target_label: str) -> None:
        super().__init__()

        feature_labels = data.columns.to_list()
        feature_labels.remove(target_label)
        
        self.X = data[feature_labels]
        self.y = data[target_label]

    def __getitem__(self, idx: int) -> tuple[pd.Series, Any]:
        return (
            th.tensor(self.X.iloc[idx, :], dtype=th.float32), 
            th.tensor(self.y.iloc[idx], dtype=th.float32)
        )
    
    def __len__(self) -> int:
        return len(self.X)

In [8]:
class MyDataModule(pl.LightningDataModule):
    def __init__(self, path: str, target_label: str, batch_size: int) -> None:
        super().__init__()
        self.path = path
        self.target_label = target_label
        self.batch_size = batch_size

    def setup(self, stage: Optional[str] = None) -> None:
        data = pd.read_csv(self.path)

        # Добавлю столбец, отражающий фактическое наличие метро для location
        has_metro = ["Москва", "Казань", "Иваново", "Кашира", "Подольск", "Люберцы", "Реутов", "Долгопрудный"]
        data["has_metro"] = data["location"].apply(lambda x: 1 if x in has_metro else 0)

        # Заполню пропуски в городах по известным станциям метро
        metro_dict = {
            "Москва": ["Авиамоторная","Автозаводская","Академическая","Александровский сад","Алексеевская","Алма-Атинская","Алтуфьево","Аннино","Арбатская","Аэропорт","Бабушкинская","Багратионовская","Баррикадная","Бауманская","Беговая","Беломорская","Белорусская","Беляево","Бибирево","Библиотека имени Ленина","Битцевский парк","Борисово","Боровицкая","Боровское шоссе","Ботанический сад","Братиславская","Бульвар адмирала Ушакова","Бульвар Дмитрия Донского","Бульвар Рокоссовского","Бунинская аллея","Бутырская","Варшавская","ВДНХ","Верхние Лихоборы","Владыкино","Водный стадион","Войковская","Волгоградский проспект","Волжская","Волоколамская","Воробьевы горы","Выставочная","Выхино","Говорово","Деловой центр","Динамо","Дмитровская","Добрынинская","Домодедовская","Достоевская","Дубровка","Жулебино","Зябликово","Измайловская","Калужская","Кантемировская","Каховская","Каширская","Киевская","Китай-город","Кожуховская","Коломенская","Коммунарка","Комсомольская","Коньково","Косино","Котельники","Красногвардейская","Краснопресненская","Красносельская","Красные ворота","Крестьянская застава","Кропоткинская","Крылатское","Кузнецкий мост","Кузьминки","Кунцевская","Курская","Кутузовская","Ленинский проспект","Лермонтовский проспект","Лесопарковая","Лефортово","Ломоносовский проспект","Лубянка","Лухмановская","Люблино","Марксистская","Марьина роща","Марьино","Маяковская","Медведково","Международная","Менделеевская","Минская","Митино","Мичуринский проспект","Молодежная","Мякинино","Нагатинская","Нагорная","Нахимовский проспект","Некрасовка","Нижегородская","Новогиреево","Новокосино","Новокузнецкая","Новопеределкино","Новослободская","Новоясеневская","Новые Черемушки","Озерная","Окружная","Окская","Октябрьская","Октябрьское поле","Ольховая","Орехово","Отрадное","Охотный ряд","Павелецкая","Парк культуры","Парк Победы","Партизанская","Первомайская","Перово","Петровский парк","Петровско-Разумовская","Печатники","Пионерская","Планерная","Площадь Ильича","Площадь Революции","Полежаевская","Полянка","Пражская","Преображенская площадь","Прокшино","Пролетарская","Проспект Вернадского","Проспект Мира","Профсоюзная","Пушкинская","Пятницкое шоссе","Раменки","Рассказовка","Речной вокзал","Рижская","Римская","Румянцево","Рязанский проспект","Савеловская","Саларьево","Свиблово","Севастопольская","Селигерская","Семеновская","Серпуховская","Славянский бульвар","Смоленская","Сокол","Сокольники","Солнцево","Спартак","Спортивная","Сретенский бульвар","Стахановская","Строгино","Студенческая","Сухаревская","Сходненская","Таганская","Тверская","Театральная","Текстильщики","Теплый Стан","Технопарк","Тимирязевская","Третьяковская","Тропарево","Трубная","Тульская","Тургеневская","Тушинская","Улица 1905 года","Улица академика Янгеля","Улица Горчакова","Улица Дмитриевского","Улица Скобелевская","Улица Старокачаловская","Университет","Филатов луг","Филевский парк","Фили","Фонвизинская","Фрунзенская","Ховрино","Хорошевская","Царицыно","Цветной бульвар","ЦСКА","Черкизовская","Чертановская","Чеховская","Чистые пруды","Чкаловская","Шаболовская","Шелепиха","Шипиловская","Шоссе Энтузиастов","Щелковская","Щукинская","Электрозаводская","Юго-Восточная","Юго-Западная","Южная","Ясенево"],
            "Казань": ["Авиастроительная","Аметьево","Горки","Козья слобода","Кремлёвская","Площадь Габдуллы Тукая","Проспект Победы","Северный вокзал","Суконная слобода","Яшьлек","Юность"]
        }
        for idx in data[data.location.isna() == True].index:
            if data.loc[idx, "underground"] in metro_dict["Москва"]:
                data.loc[idx, "location"] = "Москва"
            elif data.loc[idx, "underground"] in metro_dict["Казань"]:
                data.loc[idx, "location"] = "Казань"

        # Заполню пропуски в метраже и количестве комнат
        meters = data[["total_meters", "rooms_count"]]
        meters.loc[meters["rooms_count"] < 0, "rooms_count"] = np.nan
        meters_per_room = meters.groupby("rooms_count").median()

        data = data.merge(
            meters_per_room,
            on="rooms_count", 
            how="left", 
            suffixes=["", "_sub"]
        )
        # Метры заменю медианами по каждому количеству комнат
        data['total_meters'] = data['total_meters'].fillna(data['total_meters_sub'])
        data = data.drop(columns='total_meters_sub')

        # Заполню пропуски по этажам количеством этажей (возможно не лучший вариант) и наоборот
        floors = data[["floor", "floors_count"]]
        floor_na_idx = floors[data["floor"].isna()].index
        floors_count_na_idx = floors[data["floors_count"].isna()].index
        data.loc[floor_na_idx, "floor"] = data.loc[floor_na_idx, "floors_count"]
        data.loc[floors_count_na_idx, "floors_count"] = data.loc[floors_count_na_idx, "floor"]
        
        data = data.drop(["author", "author_type", "deal_type", "accommodation_type", "commissions", "district", "street", "house_number", "underground", "ID"], axis=1)
        # Пройдусь labelencor`ом, в cities_index сделал порядок по убыванию цены 
        cities_index = {
            "Москва": 0,
            "Подольск": 1,
            "Люберцы": 2,
            "Долгопрудный": 3,
            "Реутов": 4,
            "Казань": 5,
            "Калуга": 6,
            "Кашира": 7,
            "Рязань": 8,
            "Владимир": 9,
            "Иваново": 10,
            "Великий Новгород": 11,
            "Смоленск": 12,
            "Брянск": 13,
            "Киров": 14
        }
        data["location"] = data["location"].map(cities_index)

        # StandartScaler
        for col in data.columns:
            data.loc[:, col] = (data.loc[:, col] - data.loc[:, col].mean()) / (data.loc[:, col].std() + 1e-10)

        data = data.dropna()
        
        dset = DatasetFloors(data, self.target_label)
        self.dset_train, self.dset_valid, self.dset_test = random_split(dset, [0.7, 0.2, 0.1]) 
        
    def train_dataloader(self) -> DataLoader:
        self.setup()
        return DataLoader(self.dset_train, batch_size=self.batch_size)

    def val_dataloader(self) -> DataLoader:
        self.setup()
        return DataLoader(self.dset_valid, batch_size=self.batch_size)

    def test_dataloader(self) -> DataLoader:
        self.setup()
        return DataLoader(self.dset_test, batch_size=self.batch_size)

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import MLFlowLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
th.set_float32_matmul_precision('high')
mlf_logger = MLFlowLogger(
    experiment_name="DL_KR_logs", 
    tracking_uri="http://localhost:8080"
)

early_stop = EarlyStopping(
    monitor="loss",       
    mode="min",               
    patience=10,               
    verbose=True,
    check_on_train_epoch_end=True
)

checkpoint = ModelCheckpoint(
    monitor="R2Score",
    mode="max",
    save_top_k=1,             
    filename="best-{epoch:02d}-{R2Score:.4f}",
    save_weights_only=False
)

trainer = Trainer(
    max_epochs=1000,
    logger=mlf_logger,
    callbacks=[early_stop, checkpoint],
)

model = MyLightningModule(
    body=nn.Sequential(
        th.nn.Linear(6, 128),
        th.nn.LeakyReLU(),
        th.nn.Linear(128, 128),
        th.nn.LeakyReLU(),
        th.nn.Linear(128, 64),
        th.nn.Dropout(0.1),
        nn.BatchNorm1d(64),
        th.nn.LeakyReLU(),
        th.nn.Linear(64, 1)
    ),
    learning_rate=0.01
)

trainer.fit(
    model, 
    datamodule=MyDataModule("data/train.csv", "price_per_month", 256)
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


MlflowException: API request to http://localhost:8080/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='127.0.0.1', port=2080): Max retries exceeded with url: http://localhost:8080/api/2.0/mlflow/experiments/get-by-name?experiment_name=DL_KR_logs (Caused by ResponseError('too many 502 error responses'))